In [ ]:
# 复制 CIFAR-10 数据集到根目录
! cp -rf /bohr/cifar-p65r/v1/cifar-10-batches-py/ ./

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# 为训练和测试集定义数据变换
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 加载CIFAR-10训练和测试数据集
trainset = torchvision.datasets.CIFAR10(
    root="./", train=True, download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=100, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root="./", train=False, download=False, transform=transform)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)
# CIFAR-10的类名
classes = ('飞机', '汽车', '鸟', '猫', '鹿',
           '狗', '青蛙', '马', '船', '卡车')

In [ ]:
# 检查 GPU 是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cpu = torch.device('cpu')
print(f'Using device: {device}')

In [ ]:
import numpy as np
# 显示一批图像的函数
def imshow(img):
    img = img / 2 + 0.5  # 反归一化
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# 显示训练集中的前16个图像
dataiter = iter(trainloader)
images, labels = next(dataiter)

# 只显示前16个图像和标签
imshow(torchvision.utils.make_grid(images[:16]))
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(16)))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 实例化模型
net = Net()
net.to(device)

In [ ]:
import torch.optim as optim

def train_and_evaluate(model, train_loader, val_loader, epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            output = output.squeeze()
            target = target.squeeze()
            loss = criterion(output, target)         
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:  # 每100个小批量打印一次
                print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0
    
    # 评估模型性能
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
        accuracy = correct / total
        print(f'在10,000张测试图像上的准确率：{100 * accuracy:.2f}%')
    return accuracy

In [ ]:
# 可视化预测结果
dataiter = iter(testloader)
images, labels = next(dataiter)
images, labels = images.to(device), labels.to(device)
outputs = net(images)
_, predicted = torch.max(outputs, 1)

newimage = images.cpu().detach()

imshow(torchvision.utils.make_grid(newimage))
print('真实标签: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(10)))
print('预测标签:   ', ' '.join(f'{classes[predicted[j]]:5s}' for j in range(10)))